# Load Libraries

In [1]:
from data_utils import load_data,flatten_data
from model_utils import model_pipeline,plot_history,ensemble_predictions
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Define Participant Number

In [2]:
train_participant_num = ["C56D","C93D","C382D","C382N","C544D","C709N","C788N","P113D","P113N","P191D","P191N","P299D","P299N","P300D","P336D","P492D","P492N","P531N","P699D","P699N","P890N","P921D","P921N"]
valid_participant_num = ["C67D","C202D","C202N","C256D","C256N","P54D","P54N","P342D","P342N","P487D","P487N","P649N"]

# Load Coordinate Data

In [3]:
X_train, y_train = load_data(train_participant_num, 'train', downsampling=True,angle_energy=False,augment=False)
X_valid, y_valid = load_data(valid_participant_num, 'validation')

# Print Data Shape

In [4]:
print(X_train.shape,np.unique(y_train[:,0],return_counts=True))

print(X_valid.shape,np.unique(y_valid[:,0],return_counts=True))

(5012, 180, 70) (array([0., 1.], dtype=float32), array([1053, 3959]))
(2869, 180, 70) (array([0., 1.], dtype=float32), array([ 171, 2698]))


# Sperate Data for Random Forest and SVM

In [8]:
X_train_rf = X_train[:,:,0:66]
X_valid_rf = X_valid[:,:,0:66]
X_train_svm = X_train[:,:,66:70]
X_valid_svm = X_valid[:,:,66:70]

# Flatten Data

In [9]:
X_train_rf,y_train_rf = flatten_data(X_train_rf,y_train)
X_train_svm,y_train_svm = flatten_data(X_train_svm,y_train)

X_valid_rf,y_valid_rf = flatten_data(X_valid_rf,y_valid)
X_valid_svm,y_valid_svm = flatten_data(X_valid_svm,y_valid)

# Build Random Forest and SVM

In [10]:
rf = RandomForestClassifier()
svm = SVC(probability=True)

rf.fit(X_train_rf,y_train_rf)
svm.fit(X_train_svm,y_train_svm)

SVC(probability=True)

# Print Classification Report and Confusion Matrix for RF-SVM ensemble model with different ensemble stratagies

In [11]:
rf_pred = rf.predict_proba(X_valid_rf)
svm_pred = svm.predict_proba(X_valid_svm)

# Weighted Averaging Ensemble

In [12]:
ensemble_pred = ensemble_predictions(rf_pred,svm_pred)
print(classification_report(y_valid_rf,ensemble_pred))
print(confusion_matrix(y_valid_rf,ensemble_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      2698
           1       0.33      0.39      0.36       171

    accuracy                           0.92      2869
   macro avg       0.64      0.67      0.66      2869
weighted avg       0.92      0.92      0.92      2869

[[2560  138]
 [ 104   67]]


# Max Voting Ensemble

In [13]:
ensemble_pred = ensemble_predictions(rf_pred,svm_pred,strategy='max')
print(classification_report(y_valid_rf,ensemble_pred))
print(confusion_matrix(y_valid_rf,ensemble_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      2698
           1       0.33      0.39      0.36       171

    accuracy                           0.92      2869
   macro avg       0.64      0.67      0.66      2869
weighted avg       0.92      0.92      0.92      2869

[[2560  138]
 [ 104   67]]


# Product Ensemble

In [14]:
ensemble_pred = ensemble_predictions(rf_pred,svm_pred,strategy='product')
print(classification_report(y_valid_rf,ensemble_pred))
print(confusion_matrix(y_valid_rf,ensemble_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      2698
           1       0.33      0.39      0.36       171

    accuracy                           0.92      2869
   macro avg       0.64      0.67      0.66      2869
weighted avg       0.92      0.92      0.92      2869

[[2560  138]
 [ 104   67]]
